<a href="https://colab.research.google.com/github/tomdaler/Machine-Learning/blob/main/Wine%20Quality/2_Optimal_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [23]:
df = pd.read_csv('winequality-red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [24]:
df['quality']= df['quality']-3

In [25]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

scaled_features = df.copy()

col_names = ['fixed acidity','free sulfur dioxide','total sulfur dioxide','alcohol','pH','residual sugar']
features = scaled_features[col_names]
df_new = scale.fit_transform(features)

scaled_features[col_names] = df_new

In [26]:
X = scaled_features.drop(['quality'],axis=1)
y = scaled_features['quality']
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=2)

In [27]:
X=X.drop(['alcohol', 'sulphates','pH','density','total sulfur dioxide'], axis=1)

Explore optimal model

In [28]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from sklearn.metrics import accuracy_score

#Eliminar warnings
import warnings
warnings.simplefilter("ignore")

classifier = {
        'KNeighbors': KNeighborsClassifier(),
        'LinearSCV': LinearSVC(),
        'SVC': SVC(),
        'SGDC': SGDClassifier(),
        'DecisionTree': DecisionTreeClassifier(),
        'RandomForestClassifier': RandomForestClassifier(),
        'LogisticRegression': LogisticRegression(),
        'svm': SVC(decision_function_shape='ovo')
    }

for name, estimator in classifier.items():
    bag_class = BaggingClassifier(base_estimator=estimator, n_estimators=5).fit(train_X, train_y)
    bag_pred = bag_class.predict(test_X)
    print('Accuracy Bagging with {}:'.format(name), accuracy_score(bag_pred, test_y))
    print('')

Accuracy Bagging with KNeighbors: 0.5583333333333333

Accuracy Bagging with LinearSCV: 0.55

Accuracy Bagging with SVC: 0.58125

Accuracy Bagging with SGDC: 0.5291666666666667

Accuracy Bagging with DecisionTree: 0.5895833333333333

Accuracy Bagging with RandomForestClassifier: 0.6479166666666667

Accuracy Bagging with LogisticRegression: 0.5895833333333333

Accuracy Bagging with svm: 0.5604166666666667



In [29]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [100,120, 140],
            'max_samples': [0.7,0.9] 
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,10,20]
        }
    }
}

scores = []

from sklearn.model_selection import GridSearchCV

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.494683,"{'C': 20, 'kernel': 'linear'}"
1,random_forest,0.482204,"{'max_samples': 0.9, 'n_estimators': 140}"
2,logistic_regression,0.490917,{'C': 20}


Baggind - Cross Validation

In [30]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.7, 
    oob_score=True,
    random_state=0
)
bag_model.fit(train_X, train_y)
bag_model.oob_score_

0.6791778373547811